<a href="https://colab.research.google.com/github/luojie1024/TextClassification/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 词频-逆文件频率(TF-IDF) 

## 1. 手动实现（IF-IDF）

### 1.1 语料准备

In [23]:
# 语料
corpus = ['this is the first document',
        'this is the second second document',
        'and the third one',
        'is this the first document']

corpus = ['这 是 第一个 文档',
        '这是 第二个 文档',
        '这是 最后 一个 文档',
        '现在 没有 文档 了']

# 词袋
words_list = list()

for i in range(len(corpus)):
  # 分词,ps:英文用空格分隔
  words_list.append(corpus[i].split())

print(words_list)

[['这', '是', '第一个', '文档'], ['这是', '第二个', '文档'], ['这是', '最后', '一个', '文档'], ['现在', '没有', '文档', '了']]


### 1.2 统计词语数量


In [13]:
from collections import Counter

# 词频统计
count_list = list()

# 遍历语料
for i in range(len(words_list)):
    # 统计词频
    count = Counter(words_list[i])
    # 词频列表
    count_list.append(count)
print(count_list)

[Counter({'这': 1, '是': 1, '第一个': 1, '文档': 1}), Counter({'这是': 1, '第二个': 1, '文档': 1}), Counter({'这是': 1, '最后': 1, '一个': 1, '文档': 1}), Counter({'现在': 1, '没有': 1, '文档': 1, '了': 1})]


### 1.3 定义函数


In [14]:
import math
def tf(word, count):
    return count[word] / sum(count.values())


def idf(word, count_list):
    n_contain = sum([1 for count in count_list if word in count])
    return math.log(len(count_list) / (1 + n_contain))


def tf_idf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

### 1.4 输出结果

In [16]:
for i, count in enumerate(count_list):
    print("第 {} 个文档 TF-IDF 统计信息".format(i + 1))
    scores = {word : tf_idf(word, count, count_list) for word in count}
    sorted_word = sorted(scores.items(), key = lambda x : x[1], reverse=True)
    for word, score in sorted_word:
        print("\tword: {}, TF-IDF: {}".format(word, round(score, 5)))

第 1 个文档 TF-IDF 统计信息
	word: 这, TF-IDF: 0.17329
	word: 是, TF-IDF: 0.17329
	word: 第一个, TF-IDF: 0.17329
	word: 文档, TF-IDF: -0.05579
第 2 个文档 TF-IDF 统计信息
	word: 第二个, TF-IDF: 0.23105
	word: 这是, TF-IDF: 0.09589
	word: 文档, TF-IDF: -0.07438
第 3 个文档 TF-IDF 统计信息
	word: 最后, TF-IDF: 0.17329
	word: 一个, TF-IDF: 0.17329
	word: 这是, TF-IDF: 0.07192
	word: 文档, TF-IDF: -0.05579
第 4 个文档 TF-IDF 统计信息
	word: 现在, TF-IDF: 0.17329
	word: 没有, TF-IDF: 0.17329
	word: 了, TF-IDF: 0.17329
	word: 文档, TF-IDF: -0.05579


## 2. Gensim实现

In [17]:
words_list

[['这', '是', '第一个', '文档'],
 ['这是', '第二个', '文档'],
 ['这是', '最后', '一个', '文档'],
 ['现在', '没有', '文档', '了']]

### 2.1 获取词频

In [18]:
from gensim import corpora
# 赋给语料库中每个词(不重复的词)一个整数id
dic = corpora.Dictionary(words_list)
new_corpus = [dic.doc2bow(words) for words in words_list]
# 元组中第一个元素是词语在词典中对应的id，第二个元素是词语在文档中出现的次数
print(new_corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (4, 1), (5, 1)], [(0, 1), (5, 1), (6, 1), (7, 1)], [(0, 1), (8, 1), (9, 1), (10, 1)]]


### 2.2 查看每个词语对应的`id`

In [19]:
print(dic.token2id)

{'文档': 0, '是': 1, '第一个': 2, '这': 3, '第二个': 4, '这是': 5, '一个': 6, '最后': 7, '了': 8, '没有': 9, '现在': 10}


### 2.3 训练gensim模型并且保存它以便后面的使用


In [20]:
# 训练模型并保存
from gensim import models
tfidf = models.TfidfModel(new_corpus)
tfidf.save("tfidf.model")
# 载入模型
tfidf = models.TfidfModel.load("tfidf.model")
# 使用这个训练好的模型得到单词的tfidf值
tfidf_vec = []
for i in range(len(corpus)):
    string = corpus[i]
    string_bow = dic.doc2bow(string.lower().split())
    string_tfidf = tfidf[string_bow]
    tfidf_vec.append(string_tfidf)

# 输出 词语id与词语tfidf值
print(tfidf_vec)

[[(1, 0.5773502691896258), (2, 0.5773502691896258), (3, 0.5773502691896258)], [(4, 0.8944271909999159), (5, 0.4472135954999579)], [(5, 0.3333333333333333), (6, 0.6666666666666666), (7, 0.6666666666666666)], [(8, 0.5773502691896258), (9, 0.5773502691896258), (10, 0.5773502691896258)]]


### 2.4 句子输出

In [21]:
# 测试一个句子
test_words = "i is the first one"
string_bow = dic.doc2bow(string.lower().split())
string_tfidf = tfidf[string_bow]
print(string_tfidf)

[(8, 0.5773502691896258), (9, 0.5773502691896258), (10, 0.5773502691896258)]


## 3 使用 sklearn 算法包实现

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
tfidf_matrix = tfidf_vec.fit_transform(corpus)
# 得到语料库所有不重复的词
print(tfidf_vec.get_feature_names())
# 得到每个单词对应的id值
print(tfidf_vec.vocabulary_)
# 得到每个句子所对应的向量，向量里数字的顺序是按照词语的id顺序来的
print(tfidf_matrix.toarray())

['一个', '文档', '最后', '没有', '现在', '第一个', '第二个', '这是']
{'第一个': 5, '文档': 1, '这是': 7, '第二个': 6, '最后': 2, '一个': 0, '现在': 4, '没有': 3}
[[0.         0.46263733 0.         0.         0.         0.88654763
  0.         0.        ]
 [0.         0.37919167 0.         0.         0.         0.
  0.72664149 0.5728925 ]
 [0.58783765 0.30675807 0.58783765 0.         0.         0.
  0.         0.46345796]
 [0.         0.34618161 0.         0.66338461 0.66338461 0.
  0.         0.        ]]


# 参考
[1] https://zh.wikipedia.org/wiki/Tf-idf

[2] https://blog.csdn.net/zrc199021/article/details/53728499

[3] https://www.zybuluo.com/lianjizhe/note/1212780

[4] https://zhuanlan.zhihu.com/p/97273457